** Example of an application of the kinematic modelling to the open cluster IC2602 **


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm


from scipy import linalg
import scipy.optimize as opt

from astropy.io import fits
from astropy import wcs
from astropy.coordinates import *
from astropy import units as u
import time


from routines_mod import *
from derivatives_mod import *
from matrixOperations import *


Input file. In this case I am using the member list from van Leeuwen at al. (2017).
Note that in the paper Bravi et al. (2018) we also used a sub-set of this membership list.

In [2]:

hdu = fits.open('Data/vanLeeuwen/ic2602_all.fits')
data2602 = hdu[1].data 

In [3]:
#### The code requires ra  and dec to be in radians.

c = SkyCoord(ra = data2602['RA_ICRS']*u.degree, dec = data2602['DE_ICRS']*u.degree)
alpha, delta = c.ra.radian, c.dec.radian

In [4]:
#### Number of stars
N = len(data2602['RA_ICRS']) 

#### Observables arrays, errors, and correlation coefficients
prlx, pmra, pmdec = data2602['Plx'], data2602['pmRA'], data2602['pmDE']
e_prlx, e_pmra, e_pmdec = data2602['e_Plx'], data2602['e_pmRA'], data2602['e_pmDE']
cc_prlx_pmra, cc_prlx_pmdec, cc_pmra_pmdec = data2602['PlxpmRAcor'], data2602['PlxpmDEcor'], data2602['pmRApmDEcor'] 

#### Define three matrixes, one for the observed quantities, 
#### one for the errors and one for the correlation coefficients

obs = np.transpose(np.array([prlx,pmra,pmdec]))
sigma = np.transpose(np.array([e_prlx, e_pmra, e_pmdec]))
#cc = np.transpose(np.array([cc_prlx_pmra, cc_prlx_pmdec, cc_pmra_pmdec]))  

cc = np.zeros((N, 3))

cc[:,0] = cc_prlx_pmra 
cc[:,1] = cc_prlx_pmdec
cc[:,2] = cc_pmra_pmdec


#### Source Id 
gaiaid = data2602['Source']

Next comes possibly the **most important** part of the code, which is chosing the initial values 
for the minimization algorithm. 
The user needs to specify the initial values for the centre velocity, the velocity dispersion and 
the parallaxes.
For the parallaxes it is easy, you start with the observed ones.
For the velocity and the velocity dispersion you have to rely on your prior knowledge of the cluster.
For example, if you have an idea of the average parallax, proper motion and radial velocity 
you can convert those into cartesian coordinates. 
**The final results depend in different (complex) ways on the initial parameters. **
So if you're not sure the best way is to try with many  and then study the final parameter distribution.

Usually for nearby clusters the proper motions and the parallax are large enough to allow a decent determination of the parameters.


In [5]:
sigmav = 0.3 #### km/s
vx, vy, vz = -9.55, 16.65, -12.54 ### km/s
initial_guesses = np.concatenate((obs[:, 0],np.array([vx, vy, vz, sigmav])))

**First iteration**

There are two parameters to specify: grad e method.

You can decide whether to use or not the likelihood gradient in the minimization. If you want to use it 
write 'YES' and then in the method remember to specify a minimization method that actually requires the
likelihood gradient to be specified (e.g. Newton-CG). Otherwise type 'NO' (method = 'Nelder-Mead' or 'Powell'). The Hessian of the likelihood is also available. I used it to compute errors on the estimated quantities (Cramer Rao inequality), however it is also possible to use it for the minimization. In this case type 'HESS' and then specify 'Newton-CG' as a method.

For an overview of the minimization methods, you can have a look at: 
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

**Example with Gradient **

In [6]:
t0 = time.time()
grad = 'YES' 
method = 'Newton-CG'

result = optimizer(grad, method, initial_guesses, alpha, delta, 
                   obs, sigma, cc, N)
g_fin = g_func(result, alpha, delta, obs, sigma, cc, N)
g_fin_max = np.max(g_fin)

print( 'vx: ',result[-4], 'vy: ',result[-3], 'vz: ', result[-2], 'sigma_v: ', result[-1])
print( 'Max g value: ', g_fin_max)
print( 'Total Time:', round(time.time()-t0, 2), 's')

         Current function value: -56.173386
         Iterations: 11
         Function evaluations: 34
         Gradient evaluations: 181
         Hessian evaluations: 0
vx:  -9.97447616327 vy:  16.7720084299 vz:  -13.8312666216 sigma_v:  0.408290550096
Max g value:  71.5564386277
Total Time: 0.4 s


** Example without Gradient**

In [13]:
t0 = time.time()
grad = 'NO' 
method = 'Nelder-Mead'

result = optimizer(grad, method, initial_guesses, alpha, delta, 
                   obs, sigma, cc, N)
g_fin = g_func(result, alpha, delta, obs, sigma, cc, N)
g_fin_max = np.max(g_fin)

print( 'vx: ',result[-4], 'vy: ',result[-3], 'vz: ', result[-2], 'sigma_v: ', result[-1])
print( 'Max g value: ', g_fin_max)
print( 'Total Time:', round(time.time()-t0, 2), 's')

Optimization terminated successfully.
         Current function value: -61.284037
         Iterations: 100557
         Function evaluations: 111697
vx:  -10.3648926981 vy:  16.9409711352 vz:  -14.6925608755 sigma_v:  0.480131381874
Max g value:  51.8865027849
Total Time: 73.73 s


In this case the results after the first minimization are quite in agreement. 
The difference between the two methods consists - in this case - mainly in the execution time, which is much shorter 
using the gradient.

The reason to use the Nelder-Mead method is that sometimes the Newton-CG iterations do not converge properly or they have problems with the required precision. Nelder-Mead does not give such problems.

One way to deal with such convergence problems is to change the parameted 'xtol' in the function called 'optimizer' (see routines_mod.py).


So **my opinion** is that one should always double-check with both methods and possibly without parallax minimization (see notebook) to check whether the values obtained are reliable.

** Further iterations **

The variable $g$ is a measure of the quadratic difference between the observed and modelled quantities (i.e. parallaxes and proper motions), and it is nearly distributed as $\chi^2_2$

A $~1 \% $ significance level correponds to $g \sim 14$.
In Bravi et al. (2018) we chose $g_{threshold} = 15$.

Therefore stars with a value of $g > g_{threshold}$ are excluded one by one, and the parameters computed again for the new set of observables. 

In [7]:
g_lim = 15.
while (g_fin_max > g_lim):
    N_survive = len(g_fin[g_fin < g_fin_max])
    print( 'New N value: ', N_survive)

    alpha_sel, delta_sel = np.zeros(N_survive), np.zeros(N_survive)
    obs_sel = np.zeros((N_survive, 3))
    sigma_sel = np.zeros((N_survive, 3))
    cc_sel =  np.zeros((N_survive, 3))
    gaiaId_sel = np.zeros(N_survive)


    alpha_sel, delta_sel = alpha[g_fin < g_fin_max], delta[g_fin < g_fin_max]
    obs_sel[:, 0], obs_sel[:, 1], obs_sel[:, 2] = obs[g_fin < g_fin_max,0], obs[g_fin < g_fin_max,1], \
                                                  obs[g_fin < g_fin_max,2]
    sigma_sel[:, 0], sigma_sel[:, 1], sigma_sel[:, 2] = sigma[g_fin < g_fin_max,0], \
                                                        sigma[g_fin < g_fin_max,1], sigma[g_fin < g_fin_max,2]
    cc_sel[:, 0], cc_sel[:, 1], cc_sel[:, 2] = cc[g_fin < g_fin_max,0], cc[g_fin < g_fin_max,1], \
                                               cc[g_fin < g_fin_max,2] 
    gaiaId_sel = gaiaid[g_fin < g_fin_max]


    print( 'GaiaID of excluded star: ', gaiaid[(g_fin >= g_fin_max)])
    
    
    N = N_survive
    init_par_sel =  np.concatenate((obs_sel[:, 0], np.array([vx, vy, vz, sigmav])))
    result_sel = optimizer(grad, method, init_par_sel, 
                           alpha_sel, delta_sel, obs_sel,  sigma_sel,cc_sel, N)
    g_fin = g_func(result_sel, alpha_sel, delta_sel, 
                   obs_sel, 
                   sigma_sel,cc_sel, N)
    g_fin_max = np.max(g_fin)


    ### Compute errors on quantities.
    H =  -Nmatrix(result_sel, alpha_sel, delta_sel,obs_sel, sigma_sel,cc_sel, N)
    invH =  linalg.inv(H)
    err = np.zeros(N+4)
    for i in range(N+4):
        err[i] = np.sqrt(-invH[i,i])

    ### Upload initial values
    alpha, delta = alpha_sel,delta_sel
    obs, sigma, cc = obs_sel, sigma_sel, cc_sel
    gaiaid = gaiaId_sel


    ### Print
    U =  Ulike(result_sel,  alpha_sel, delta_sel, obs_sel, sigma_sel, cc_sel, N)
    print('Max g value: ', g_fin_max)
    print( ' Likelihood: ', U)
    print( 'vx:',round(result_sel[-4],2),'km/s, ', 'vy:',round(result_sel[-3],2),'km/s, ', 
          'vz:', round(result_sel[-2],2),'km/s, ', 'sigma_v:',round( result_sel[-1],2), 'km/s, ',)
    print('e_vx:', round(err[-4], 2),'km/s, ', 'e_vy: ', round(err[-3], 2),'km/s, ', 
          'e_vz:', round(err[-2],2),'km/s, ', 'e_sigmav: ', round(err[-1],2), 'km/s ')
    
        
        


New N value:  65
GaiaID of excluded star:  [5252084918078077952]
         Current function value: -71.579090
         Iterations: 3
         Function evaluations: 4
         Gradient evaluations: 2781
         Hessian evaluations: 0
Max g value:  20.339403888
 Likelihood:  -71.57908968602385
vx: -9.57 km/s,  vy: 16.52 km/s,  vz: -12.55 km/s,  sigma_v: 0.34 km/s, 
e_vx: 0.74 km/s,  e_vy:  0.3 km/s,  e_vz: 1.65 km/s,  e_sigmav:  0.03 km/s 
New N value:  64
GaiaID of excluded star:  [5239689092704584704]
         Current function value: -142.266390
         Iterations: 8
         Function evaluations: 36
         Gradient evaluations: 2236
         Hessian evaluations: 0
Max g value:  22.7587702687
 Likelihood:  -142.26638951693312
vx: -10.67 km/s,  vy: 16.95 km/s,  vz: -15.18 km/s,  sigma_v: 0.15 km/s, 
e_vx: 0.44 km/s,  e_vy:  0.2 km/s,  e_vz: 0.98 km/s,  e_sigmav:  0.02 km/s 
New N value:  63
GaiaID of excluded star:  [5251495957802232448]
         Current function value: -157.601996
 

**Final parameters using Nelder-Mead:**

$v_x = -11.72  \pm 0.53 $ km/s

$v_y = 17.31 \pm  0.23  $ km/s

$v_z = -17.53 \pm  1.18  $ km/s

$\sigma_v= 0.2 \pm   0.02 $ km/s

** Final parameters from Newton-CG: **


$v_x = -11.69  \pm 0.62 $ km/s

$v_y = 17.31 \pm  0.26  $ km/s

$v_z = -17.45 \pm  1.38  $ km/s

$\sigma_v= 0.12 \pm   0.01 $ km/s

** Remarks**

The estimated parameters are compatible. With Newton-CG there are some warnings related to precision/convergence loss. So my suggestion is to always double - check with the two methods the parameters obtained (and possibly also with other methods, see e.g. this notebook)

**Last step - unbiased velocity dispersion**

Now the velocity dispersion obtained with this method is underestimated. Therefore we use the formulae in 
Lindegren et al. (2000) appendix A.4 to compute a better value for $\sigma_v$.

In [13]:
_A = auKmYearPerSec
v = [result_sel[-4], result_sel[-3], result_sel[-2]]  ### These are vx, vy, vz
prlx_est = result_sel[:-4] ### Estimated parallaxes


p, q, r = normalTriad(alpha_sel, delta_sel) ### Remember that coordinates must be in radians.
pmra_est = np.dot(np.transpose(p),v)*prlx_est/_A
pmde_est = np.dot(np.transpose(q),v)*prlx_est/_A

#### Observed quantities
prlx, pmra, pmdec = obs_sel[:,0], obs_sel[:,1], obs_sel[:,2]  


### Define new coordinate system, as in appendix A.4 of Lindegren+00
N = len(alpha_sel)
k_perp = []

for i in range(N): 
    k_perp_ = np.cross(r.T[i], v)
    k_perp_ = k_perp_/(k_perp_[0]**2.+k_perp_[1]**2.+k_perp_[2]**2.)**(0.5) ### Eq. A.19
    k_perp.append(k_perp_)

k_par = []

for i in range(N):
    k_par_ = np.cross(r.T[i], k_perp[i])  ### Eq. A.19
    k_par.append(k_par_)
    
h = []
for i in range(N):
    h_ = np.dot(np.asarray([np.zeros(3), p.T[i], q.T[i]]), k_perp[i])   ### Eq. A.20
    h.append(h_)
    
eta = []

for i in range(N):
    eta_ = np.dot(h[i], [prlx[i] - prlx_est[i],
                         pmra[i] - pmra_est[i], 
                         pmdec[i] - pmde_est[i]])   
    eta_ = eta_*_A/prlx_est[i]  ### Eq. A.21
    eta.append(eta_)
    


### In this covariance matrix only the proper motion errors are taken into account
C = np.zeros((3,3,N),dtype=np.float64)
C[0, 0, :] = np.zeros(N)
C[1,1,:] = (sigma_sel[:, 1])**2.
C[2,2,:] = (sigma_sel[:, 2])**2.
plxPmRa, plxPmDec, pmRapmDec = np.zeros(N), np.zeros(N), np.zeros(N)
pmRapmDec =  cc_sel[:, 2] 
C[0,1,:], C[0,2,:] =plxPmRa*sigma_sel[:, 0]*sigma_sel[:, 1], plxPmDec*sigma_sel[:, 0]*sigma_sel[:, 2]
C[1,0,:], C[1,2,:] = plxPmRa*sigma_sel[:, 0]*sigma_sel[:, 1], pmRapmDec*sigma_sel[:, 1]*sigma_sel[:, 2]
C[2,0,:], C[2,1,:] = plxPmDec*sigma_sel[:, 0]*sigma_sel[:, 2], pmRapmDec*sigma_sel[:, 1]*sigma_sel[:, 2]


sigma_eta = []
for i in range(N):
    sigma_eta_ = np.dot(np.dot(h[i],C[:,:,i]), h[i])
    sigma_eta_ = _A*(sigma_eta_)**0.5/prlx_est[i]  ### Eq. A.22
    sigma_eta.append(sigma_eta_)
    

### Compute the dispersion perpendicular to the radial velocity direction
### The function is described in Eq. A. 23
def func(observations, x):
    e, sigma_e = observations
    f = np.zeros(N)
    f[:] = (e**2. - x**2 - sigma_e**2.)/(x**2. + sigma_e**2.)**2.
    return np.sum(f)

sigma_perp, cov = opt.curve_fit(func, np.array([eta, sigma_eta]), np.zeros(N), p0 = 0.3, maxfev = 2000)
sigma_perp_err_ = (np.asarray(sigma_perp)**2.+ np.asarray(sigma_eta)**2)**(-2.)
sigma_perp_err = (2*np.abs(sigma_perp)*np.sum(sigma_perp_err_))**(-0.5) ### Eq. A.24

print('Velocity dispersion:', np.round(sigma_perp, 2), 'km/s')
print('Error on velocity dispersion:', np.round(sigma_perp_err, 2), 'km/s')

Velocity dispersion: [ 0.25] km/s
Error on velocity dispersion: [ 0.02] km/s
